Aprèn a realitzar web scraping.

### Nivell 1
**- Exercici 1:**

Realitza web scraping d'una pàgina de la borsa de Madrid (https://www.bolsamadrid.es) utilitzant BeautifulSoup i Selenium.

In [213]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd
pd.set_option('display.max_columns', None)

In [214]:
# asignar url
url = ('https://www.bolsamadrid.es')
page = requests.get(url)

In [215]:
soup = BeautifulSoup(page.content, 'html.parser')

In [216]:
#obtención de la ruta al listado completo de acciones
results = soup.find('a', string='Acciones')
results = results.get('href')
print(results)

/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000


In [217]:
#link a la lista de las acciones
link = (url+results)
print(link)

https://www.bolsamadrid.es/esp/aspx/Mercados/Precios.aspx?indice=ESI100000000


In [218]:
html = requests.get(link)
soup = BeautifulSoup(html_acciones.content, 'html.parser')

In [219]:
#obtenemos los nombres de las columnas
column_names = soup.find(id='ctl00_Contenido_tblAcciones')
column_names.find_all('th')

[<th scope="col">Nombre</th>,
 <th scope="col">Últ.</th>,
 <th scope="col">% Dif.</th>,
 <th scope="col">Máx.</th>,
 <th scope="col">Mín.</th>,
 <th scope="col">Volumen</th>,
 <th scope="col">Efectivo (miles €)</th>,
 <th scope="col">Fecha</th>,
 <th class="Ult" scope="col">Hora</th>]

In [220]:
#se crea un array para almacenar los nombres de las columnas
array = []
for element in column_names.find_all("th"):
    array.append(element.get_text())
print(array)

['Nombre', 'Últ.', '% Dif.', 'Máx.', 'Mín.', 'Volumen', 'Efectivo (miles €)', 'Fecha', 'Hora']


In [221]:
#obtención de los valores de los elementos que llenarán el array
stocks = []
for element in column_names.find_all("td"):
    stocks.append(element.text)
print (stocks)

['ACCIONA', '180,1000', '0,61', '181,1000', '178,6000', '12.102', '2.178,40', '04/07/2022', '11:06:30', 'ACCIONA ENER', '38,0600', '1,28', '38,3000', '37,8400', '29.402', '1.119,37', '04/07/2022', '11:06:17', 'ACERINOX', '8,6020', '-1,38', '8,8600', '8,5840', '346.897', '3.020,63', '04/07/2022', '11:06:26', 'ACS', '21,7300', '0,65', '21,8400', '21,5800', '163.729', '3.552,45', '04/07/2022', '11:06:33', 'AENA', '127,1500', '2,66', '127,1500', '124,5500', '21.286', '2.684,12', '04/07/2022', '11:06:30', 'AMADEUS', '53,0600', '0,19', '53,7400', '53,0200', '49.611', '2.644,70', '04/07/2022', '11:06:46', 'ARCELORMIT.', '21,7000', '0,51', '22,0000', '21,5750', '57.286', '1.247,10', '04/07/2022', '11:06:25', 'B.SANTANDER', '2,6710', '-0,48', '2,6980', '2,6615', '2.867.087', '7.681,44', '04/07/2022', '11:06:46', 'BA.SABADELL', '0,7378', '-2,97', '0,7640', '0,7312', '12.846.357', '9.577,56', '04/07/2022', '11:06:29', 'BANKINTER', '5,7640', '-2,64', '5,9480', '5,7020', '637.686', '3.704,97', '04/

In [222]:
# se añaden los valores al array mediante un diccionario que asigna los valores
# a la clave correspondiente
fill_stocks ={ }
for i in range(9):
    fill_stocks[array[i]]= stocks [i::9]

In [223]:
# creación del DF con el listado iterable del IBEX 35
df = pd.DataFrame(fill_stocks)
print(df)

          Nombre      Últ. % Dif.      Máx.      Mín.     Volumen  \
0        ACCIONA  180,1000   0,61  181,1000  178,6000      12.102   
1   ACCIONA ENER   38,0600   1,28   38,3000   37,8400      29.402   
2       ACERINOX    8,6020  -1,38    8,8600    8,5840     346.897   
3            ACS   21,7300   0,65   21,8400   21,5800     163.729   
4           AENA  127,1500   2,66  127,1500  124,5500      21.286   
5        AMADEUS   53,0600   0,19   53,7400   53,0200      49.611   
6    ARCELORMIT.   21,7000   0,51   22,0000   21,5750      57.286   
7    B.SANTANDER    2,6710  -0,48    2,6980    2,6615   2.867.087   
8    BA.SABADELL    0,7378  -2,97    0,7640    0,7312  12.846.357   
9      BANKINTER    5,7640  -2,64    5,9480    5,7020     637.686   
10          BBVA    4,3385   0,58    4,3680    4,3170   2.606.043   
11     CAIXABANK    3,2530  -1,81    3,3220    3,2200   3.466.757   
12       CELLNEX   38,1600   0,00   38,4800   37,8900     110.997   
13        ENAGAS   21,5800   1,46 

A continuanción creo un programa que permita extraer el valor de las acciones de una empresa específica en el momento de la búsuqeda

In [224]:
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By

driver_path = Service('/Applications/chromedriver')
driver = webdriver.Chrome(service = driver_path)
options = webdriver.ChromeOptions()
options.add_argument('--start-maximized')
driver.get('https://www.bolsamadrid.es')

# selección de la casilla de búsqeuda medainte xpath
search = driver.find_element(By.XPATH, '//*[@id="formBusq"]/input')
#intruducción te texto. Aquí se puede cambiar la empresa.
search.send_keys('telefonica')

#confirmación de la empresa en el listado de bísqueda
enter = driver.find_element(By.XPATH, '//*[@id="formBusq"]/span/a/span')
enter.click()
acciona = driver.find_element(By.XPATH, '//*[@id="ctl00_Contenido_tblEmisoras"]/tbody/tr[2]/td/a')
acciona.click()

#obtención de valor de acciones.
date = driver.find_element(By.XPATH, '//*[@id="ctl00_Contenido_tblPrecios"]/tbody/tr[2]/td[1]')
time = driver.find_element(By.XPATH, '//*[@id="ctl00_Contenido_tblPrecios"]/tbody/tr[2]/td[2]')
closing = driver.find_element(By.XPATH, '//*[@id="ctl00_Contenido_tblPrecios"]/tbody/tr[2]/td[6]')
print("Valor Telefonica: ", closing.text)
print("Fecha: ", date.text)
print("Hora: ", time.text)
driver.quit()

#driver.find_elements_by_xpath('//*[@id="formBusq"]/input')

Valor Telefonica:  4,9200
Fecha:  04/07/2022
Hora:  11:42:20


### Nivell 2
**- Exercici 2:**
Documenta en un word el teu conjunt de dades generat amb la informació que tenen els diferents arxius de Kaggle.



## Nivell 3
**- Exercici 3**
Tria una página web que tu vulguis i realitza web scraping mitjançant la llibreria Scrapy. 

